# <div style="color:#fff;display:fill;border-radius:10px;background-color:#ffcc00;text-align:left;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%"> . | Resumo</div>

<p style="font-size:15px; font-family:verdana; line-height: 1.7em; margin-left:20px">
    Neste Notebook vou realizar uma breve análise dos dados do desafio Wallmart Sales e fornecerei um modelo para previsão de Weekly_Sales para fins de análise. </p>

# <div style="color:#fff;display:fill;border-radius:10px;background-color:#ffcc00;text-align:left;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%"> . | Conteúdo</div>

* [Libs](#-.-|-Libs)
* [1-Carregamento dos Dados](#-1-|-Carga-de-dados)
* [2-EDA](#-2-|-An%C3%A1lise-Explorat%C3%B3ria-dos-dados-(EDA))
    * [2.1-Atributos Numéricos](#-2.1-|-Num%C3%A9ricos)
    * [2.2-Atributos Categóricos](#-2.2-|-Categ%C3%B3ricos)
    * [2.3-Detalhamento de Weekly Sales](#-2.3-|-Drilldown-Weekly_Sales)
    * [2.4-Dept 65](#-2.4-|-Estudo-do-caso-do-Dept-65)
    * [2.5-Correlação entre Atributos](#-2.5-|-Heatmap-e-correla%C3%A7%C3%A3o-entre-features)
    * [2.6-Análise de Vendas](#-2.6-|-An%C3%A1lise-de-Vendas)
* [3-Modelagem e Submissão](#-3-|-Modelo)

<a id="section-one"></a>
# <div style="color:#fff;display:fill;border-radius:10px;background-color:#ffcc00;text-align:left;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%"> . | Libs</div>

In [ ]:
import os
import numpy as np
import pandas as pd

import shap
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()

from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
#from pycaret.regression import *


import warnings
warnings.filterwarnings('ignore') 

In [ ]:
input_dfs = dict()
for dirname, _, filenames in os.walk(r'../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        input_dfs[filename.split('.')[0]] = pd.read_csv(os.path.join(dirname, filename))

<br><br>
# <div style="color:#fff;display:fill;border-radius:10px;background-color:#ffcc00;text-align:left;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%"> 1 | Carga de dados</div>

In [ ]:
# Cruzar os dados das lojas com as features e os dados de treino para obter todos os atributos da base de treino
df = input_dfs['stores'].merge(input_dfs['features'], how='inner').merge(input_dfs['train'], how='inner')

In [ ]:
# Cruzar os dados das lojas com as features e os dados de teste para obter todos os atributos da base de teste
test_df = input_dfs['stores'].merge(input_dfs['features'], how='inner').merge(input_dfs['test'], how='inner')

<br><br>
# <div style="color:#fff;display:fill;border-radius:10px;background-color:#ffcc00;text-align:left;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%"> 2 | Análise Exploratória dos dados (EDA)</div>

<p style="font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
EDA é uma das partes mais importantes do processo, pois fornece ideias sobre a relação entre atributos ou sobre o comportamento dos dados dentre outros.

In [ ]:
df.info()

<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
 <div>Type tem o perfil de dados categóricos (tipo de estabelecimento)</div>
 <div>Date Precisa ser convertido. Será destrinchado em atributos de Ano, Mês, Semana e dia para análises mais precisas</div>
 <div>Dept temo perfil de dados categóricos (o departamento). Embora seja originalmente numérico, não há relação de grandeza ou ordenação entre um número e outro</div>
</div>

In [ ]:
df.Type = df.Type.astype('category')
df.Dept = df.Dept.astype('category')
df.Date = pd.to_datetime(df.Date)
df['Day'] = df['Date'].dt.day
df['Week'] = df['Date'].dt.week
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df.drop(columns=['Date'], inplace=True)

In [ ]:
# Novo Perfil
df.info()

In [ ]:
for tp in df.dtypes.unique():
    print(tp)

<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
 <div>Há atributos 3 atributos categóricos e 16 numéricos.</div>
 <div>Os categóricos possuem índices de categorias diferentes</div>
</div>

In [ ]:
# Criação de um dataframe agrupado por semanas
df_semanas = df.groupby('Week').sum()

<br><br>
## <div style="color:#fff;display:fill;border-radius:10px;background-color:#ffcc0080;text-align:left;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%"> 2.1 | Numéricos</div>

<p style="font-size:15px; font-family:verdana; line-height: 1.7em; margin-left:20px">   
Detalhamento de atributos numéricos </p>

In [ ]:
df.select_dtypes(include=['int64','float64']).describe().T.style.bar(subset=['mean','max'], color='#ffcc00')\
                            .background_gradient(subset=['min','max'], cmap='Reds')\
                            .background_gradient(subset=['75%'], cmap='YlGn')

<br><br>
## <div style="color:#fff;display:fill;border-radius:10px;background-color:#ffcc0080;text-align:left;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%"> 2.2 | Categóricos</div>

<p style="font-size:15px; font-family:verdana; line-height: 1.7em; margin-left:20px">   
Detalhamento de atributos categóricos </p>

In [ ]:
df.select_dtypes(include=['category']).describe().T

In [ ]:
df.select_dtypes(include=['bool']).describe().T

<br><br>
## <div style="color:#fff;display:fill;border-radius:10px;background-color:#ffcc0080;text-align:left;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%"> 2.3 | Drilldown Weekly_Sales</div>

<p style="font-size:15px; font-family:verdana; line-height: 1.7em; margin-left:20px">   
Detalhamento do comportamento de Weekly_Sales</p>

In [ ]:
df[df.Weekly_Sales > 20205.852500].shape

In [ ]:
df[df.Weekly_Sales > 20205.852500].select_dtypes(include=['int64','float64']).describe().T.style.bar(subset=['mean','max'], color='#ffcc00')\
                            .background_gradient(subset=['min','max'], cmap='Reds')\
                            .background_gradient(subset=['75%'], cmap='YlGn')

In [ ]:
q4_sales = df[df.Weekly_Sales > 20205.852500].Weekly_Sales.sum()/1000000
df_q3_sales = df[df.Weekly_Sales <= 20205.852500].Weekly_Sales.sum()/1000000
print(f"Total de vendas do top 25%: {q4_sales} Milhões")
print(f"Total de vendas dos demais 75%{df_q3_sales} Milhões")
print(f"Percentual de vendas do top 25%: {round(100*q4_sales/(df_q3_sales+q4_sales),3)}%")

<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
 <div>Dos 421570 registros, 105393 estão no quartil superior (q4) de Weekly Sales</div>
 <div>Este top 25% das registros de Weekly Sales representam 71.677% do total das vendas</div>
</div>

In [ ]:
print(f"Depts ao todo: {len(df.Dept.unique())}")
print(f"Depts no top 25%: {len(df[df.Weekly_Sales > 20205.852500].Dept.unique())}")
print(f"Depts nos demais 75%: {len(df[df.Weekly_Sales <= 20205.852500].Dept.unique())}")

In [ ]:
print(f"Única diferença entre \"75% low\" e o total de Depts: {set(df.Dept.unique()) - set(df[df.Weekly_Sales <= 20205.852500].Dept.unique())}")

<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
 <div>Um único departamento nunca faz parte dos 75% menos expressivos</div>
 <div>Todas as vendas semanais do departamento 65 estão no top 25%</div>
</div>

## <div style="color:#fff;display:fill;border-radius:10px;background-color:#ffcc0080;text-align:left;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%"> 2.4 | Estudo do caso do Dept 65</div>

<p style="font-size:15px; font-family:verdana; line-height: 1.7em; margin-left:20px">   
Detalhamento do comportamento do departamento top 25%</p>

<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
 <div>Loja 34, e só a 34</div>
 <div>Type A</div>
 <div>Size 158114.000000</div>
 <div>CPI quase não varia (126.064000 ~ 131.193097)</div>
 <div>Dez vendas em feriados</div>
 <div>A loja 34 tem 78 departamentos</div>
 <div>Nenhuma loja possui apenas um departamento</div>
 <div>65 foi o décimo quinto Dept que menos vendeu ao todo</div>
 <div>Departamento que mais vendeu: 92, 483.94 milhões</div>
</div>

In [ ]:
# nenhuma loja possui apenas um departamento
for lj in [*df.Store.unique()]:
    if len(df[df.Store == lj].Dept.unique()) == 1:
        print(lj)

In [ ]:
df_65 = df[df.Dept == 65]

In [ ]:
df_65.select_dtypes(include=['int64','float64']).describe().T.style.bar(subset=['mean','max'], color='#ffcc00')\
                            .background_gradient(subset=['min','max'], cmap='Reds')\
                            .background_gradient(subset=['75%'], cmap='YlGn')

In [ ]:
# quantidade de departamentos da loja 34
df[df.Store == 34].Dept.unique()

In [ ]:
df_65.select_dtypes(include=['category']).describe().T

In [ ]:
df_65.Type.value_counts()

In [ ]:
df_65.select_dtypes(include=['bool']).describe().T

In [ ]:
df_65.IsHoliday.value_counts()

In [ ]:
sales_65 = df[df.Dept == 65].Weekly_Sales.sum()/1000000
df_sales = df[df.Dept != 65].Weekly_Sales.sum()/1000000
print(f"Total de vendas do Dept 65: {sales_65} Milhões")
print(f"Total de vendas dos demais: {df_sales} Milhões")
print(f"Percentual de vendas do Dept {round(100*sales_65/(sales_65+df_sales),3)}%")

In [ ]:
dept_sales = dict()
for dp in df.Dept.unique():
    dept_sales[dp] = df[df.Dept == dp].Weekly_Sales.sum()/1000000
dept_sales = {k: v for k, v in sorted(dept_sales.items(), key=lambda item: item[1])}

In [ ]:
# décimo quinto que menos vendeu
[*dept_sales][:15]

In [ ]:
print([*dept_sales][14], dept_sales[[*dept_sales][14]])

In [ ]:
print([*dept_sales][-1], dept_sales[[*dept_sales][-1]])

<a id="subsection-three-three"></a>
## <div style="color:#fff;display:fill;border-radius:10px;background-color:#ffcc0080;text-align:left;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%;margin-left:10px"> 2.5 | Heatmap e correlação entre features</div>

In [ ]:
cores = px.colors.qualitative.Safe

In [ ]:
corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
df_mask = corr.mask(mask).round(2)

fig = ff.create_annotated_heatmap(z=df_mask.to_numpy(), 
                                  x=df_mask.columns.tolist(),
                                  y=df_mask.columns.tolist(),
                                  colorscale=px.colors.diverging.RdYlBu_r,
                                  hoverinfo="none", 
                                  showscale=True, ygap=1, xgap=1
                                 )

fig.update_xaxes(side="bottom")

fig.update_layout(
    title_text='Heatmap', 
    title_x=0.5, 
    width=1000, 
    height=1000,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    yaxis_autorange='reversed',
    template='plotly_white'
)

for i in range(len(fig.layout.annotations)):
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

fig.show()

In [ ]:
corr_df = pd.DataFrame(df.corr().Weekly_Sales).drop(axis='index', labels='Weekly_Sales')
corr_df.sort_values(by='Weekly_Sales', ascending = False, inplace=True)
bar = px.bar(corr_df, x = corr_df.index, y = 'Weekly_Sales', color=corr_df.index, labels={'index':'Featues'},
             title='Correlação dos atributos com as vendas semanais',color_discrete_sequence=cores)
bar.update_traces(showlegend=False)

## <div style="color:#fff;display:fill;border-radius:10px;background-color:#ffcc0080;text-align:left;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%"> 2.6 | Análise de Vendas</div>

<p style="font-size:15px; font-family:verdana; line-height: 1.7em; margin-left:20px">   
Um pouco mais sobre as vendas</p>

In [ ]:
px.line( data_frame = df_semanas, x = df_semanas.index, y = 'Weekly_Sales', 
        labels = {'Weekly_Sales' : 'Weekly Sales', 'x' : 'Semanas' }, 
        title = 'Sales over weeks', color_discrete_sequence=cores)

<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
  <div>As vendas são suficientemente estáveis, porém há uma queda na semana 44 (algo entre outubro e novembro), uma retomada seguida de queda no Thanksgiving ("Dia de Ação de Graças"), e um aumento no Natal.</div>
  <div>Há um hiato de vendas durantes as primeiras semanas do ano (Janeiro)</div>
</div>

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( x = df_semanas.index, y = df_semanas['MarkDown1'], name = 'MarkDown1', mode = 'lines') )
fig.add_trace(go.Scatter( x = df_semanas.index, y = df_semanas['MarkDown2'], name = 'MarkDown2', mode = 'lines') )
fig.add_trace(go.Scatter( x = df_semanas.index, y = df_semanas['MarkDown3'], name = 'MarkDown3', mode = 'lines') )
fig.add_trace(go.Scatter( x = df_semanas.index, y = df_semanas['MarkDown4'], name = 'MarkDown4', mode = 'lines') )
fig.add_trace(go.Scatter( x = df_semanas.index, y = df_semanas['MarkDown5'], name = 'MarkDown5', mode = 'lines') )
fig.add_trace(go.Scatter( x = df_semanas.index, y = df_semanas['Weekly_Sales'], name = 'Weekly Sales', mode = 'lines') )
fig.update_layout(title = "Sales vs Markdown's", xaxis_title = 'Weeks')

<p style="font-size:20px; font-family:verdana; line-height: 1.7em; margin-left:20px">
<b> Comparação de média de vendas ao longo dos anos
         </b></p>

In [ ]:
weekly_sales = df.groupby(['Year','Week'], as_index = False).agg({'Weekly_Sales': ['mean', 'median']})
weekly_sales2010 = df.loc[df['Year']==2010].groupby(['Week']).agg({'Weekly_Sales': ['mean', 'median']})
weekly_sales2011 = df.loc[df['Year']==2011].groupby(['Week']).agg({'Weekly_Sales': ['mean', 'median']})
weekly_sales2012 = df.loc[df['Year']==2012].groupby(['Week']).agg({'Weekly_Sales': ['mean', 'median']})

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( x = weekly_sales2010['Weekly_Sales']['mean'].index, y = weekly_sales2010['Weekly_Sales']['mean'], name = 'Média de 2010', mode = 'lines') )
fig.add_trace(go.Scatter( x = weekly_sales2011['Weekly_Sales']['mean'].index, y = weekly_sales2011['Weekly_Sales']['mean'], name = 'Média de 2011', mode = 'lines') )
fig.add_trace(go.Scatter( x = weekly_sales2012['Weekly_Sales']['mean'].index, y = weekly_sales2012['Weekly_Sales']['mean'], name = 'Média de 2012', mode = 'lines') )
fig.add_annotation(text="Thanskgiving", x=47, y=25000, showarrow=False)
fig.add_annotation(text="Natal", x=51, y=29000, showarrow=False)
fig.update_layout(title = 'Sales 2010, 2011, 2012', xaxis_title = 'Semana')

<p style="font-size:15px; font-family:verdana; line-height: 1.7em;">
    <br><br>
    
<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
  <div>A influência dos feriados do Natal e Thanksgiving é evidente.
</div>
</div>

In [ ]:
# Conversão da temperatura para Celsius
df['Temperature'] = df['Temperature'].apply(lambda x :  (x - 32) * 5/9)

In [ ]:
px.scatter(df, x='Temperature', y ='Weekly_Sales', color='IsHoliday', 
           title = 'Temperature e Vendas por feriado', color_discrete_sequence=cores)

<p style="font-size:15px; font-family:verdana; line-height: 1.7em">
    <br><br>
<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
 <div>Há uma correlação entre frio e vendas, observando que os dados dos EUA onde as temperaturas são mais baixas.
</div>
</div>

In [ ]:
px.scatter(df, x='Fuel_Price', y ='Weekly_Sales', color='IsHoliday', 
           title='Preço do combustível e Vendas por feriado',color_discrete_sequence=cores)

<p style="font-size:15px; font-family:verdana; line-height: 1.7em">
    <br><br>
<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
 <div>Não há um padrão muito claro, mas os preços de combustível mais altos induzem menos vendas.
</div>
</div>

In [ ]:
px.scatter(df, x='Unemployment', y ='Weekly_Sales', color='IsHoliday', 
           title='Taxa de desemprego e Vendas por feriado',color_discrete_sequence=cores)

<p style="font-size:15px; font-family:verdana; line-height: 1.7em">
    <br><br>
<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
 <div>Como esperado, quanto mais baixo o desemprego, maiores as vendas.
</div>
</div>

In [ ]:
sizes= df.groupby('Size').mean()
px.line(sizes, x = sizes.index, y = sizes.Weekly_Sales, 
        title='Tamanho das lojas e Vendas por feriado',color_discrete_sequence=cores)

<p style="font-size:15px; font-family:verdana; line-height: 1.7em">
    <br><br>
<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
 <div>O tamanho das lojas tem grande influência nas vendas.
</div>
</div>

In [ ]:
store_type = pd.concat([input_dfs['stores']['Type'], input_dfs['stores']['Size']], axis=1)
px.box(store_type, x='Type', y='Size', color='Type', 
       title='Tamanho das Lojas e Tipo das Lojas',color_discrete_sequence=cores)

In [ ]:
store_sale = pd.concat([input_dfs['stores']['Type'], df['Weekly_Sales']], axis=1)
px.box(store_sale.dropna(), x='Type', y='Weekly_Sales', color='Type', 
       title='Tipo de loja e Vendas por feriado',color_discrete_sequence=cores)

<p style="font-size:15px; font-family:verdana; line-height: 1.7em">
    <br><br>
<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
 <div>As lojas do tipo C são poucas, se comparadas aos demais tipos.</div>
 <div>E embora em menor quantidade, as lojas de tipo C são as que mais vendem.</div>
</div>

In [ ]:
depts= df.groupby('Dept').mean().sort_values(by='Weekly_Sales', ascending='False')
bar=px.bar(depts, x = depts.index, y =  depts.Weekly_Sales, 
           title='Vendas por Departamento',color=depts.Weekly_Sales)
bar.update_layout(barmode='group', xaxis={'categoryorder':'total descending'})

<p style="font-size:15px; font-family:verdana; line-height: 1.7em">
    <br><br>
<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
 <div>Alguns departamentos são bem mais expressivos em termos de vendas do que outros.</div>
 <div>Talvez uma clusterização por grupos de departamentos trouxesse maiores insights.</div>
</div>

<br><br>
# <div style="color:#fff;display:fill;border-radius:10px;background-color:#ffcc00;text-align:left;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%"> 3 | Modelo</div>

<p style="font-size:15px; font-family:verdana; line-height: 1.7em; margin-left:20px">   
Com tratamento mais acertivo dos dados </p>

In [ ]:
df.Type = df.Type.astype('category').cat.codes
df.Dept = df.Dept.astype('category').cat.codes

for c in df:
    if df[c].isna().sum():
        df[c] = df[c].fillna(0)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='Weekly_Sales'), df.Weekly_Sales, test_size=0.25, random_state=42)

In [ ]:
regr = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='squared_error',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=-1, oob_score=False,
                       random_state=123, verbose=0, warm_start=False)
regr.fit(X_train, y_train)

In [ ]:
rf_shap_values = shap.KernelExplainer(regr.predict,shap.sample(X_test,250))

In [ ]:
shap.initjs()
shap.summary_plot( rf_shap_values.shap_values(shap.sample(X_test,250)), shap.sample(X_test,250) )

<p style="font-size:15px; font-family:verdana; line-height: 1.7em;">
    <br><br>
    
<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
  <div>Departamento é o atributo mais importante mas, como esperado, não há uma relação clara sobre a tendência de crescimento do atributo e as Vendas.</div>
  <div>O comportamento do atributo "tamanho da loja" indica que uma relação linearmente proporcional às vendas</div>
  <div>O atributo "tipo da loja" se mostrou importante e, embora um tanto quanto confuso, os gráficos anteriores ajudam a interpretá-lo</div>
</div>

In [ ]:
features = pd.DataFrame(regr.feature_importances_,X_train.columns)
features.sort_values(by=[0], ascending=False, inplace=True)
features

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, regr.predict(X_test))

<p style="font-size:15px; font-family:verdana; line-height: 1.7em">
    <br><br>
<div class="alert alert-info" style="border-radius:10px; font-size:15px; font-family:verdana; line-height: 1.7em;margin-left:20px">
<div>O modelo precisa de ajustes.</div>
</div>

In [ ]:
test_df.Type = test_df.Type.astype('category')
test_df.Dept = test_df.Dept.astype('category')
test_df.Date = pd.to_datetime(test_df.Date)
test_df['Day'] = test_df['Date'].dt.day
test_df['Week'] = test_df['Date'].dt.week
test_df['Month'] = test_df['Date'].dt.month
test_df['Year'] = test_df['Date'].dt.year
# Conversão da temperatura para Celsius
test_df['Temperature'] = test_df['Temperature'].apply(lambda x :  (x - 32) * 5/9)

test_df.Type = test_df.Type.astype('category').cat.codes
test_df.Dept = test_df.Dept.astype('category').cat.codes

for c in test_df:
    if test_df[c].isna().sum():
        test_df[c] = test_df[c].fillna(0)


In [ ]:
predictions = regr.predict(test_df.drop(columns='Date'))

In [ ]:
input_dfs['sampleSubmission']['Weekly_Sales'] = predictions
input_dfs['sampleSubmission'].to_csv('sampleSubmission.csv',index=False)

submission = pd.DataFrame({'Id':[ "_".join([str(r.Store),str(int(r.Dept+1)),str(pd.to_datetime(r.Date).date())]) for _,r in test_df.iterrows() ], 'Weekly_Sales':predictions})
submission.to_csv('submission.csv',index=False)